In [1]:
import pandas as pd
import numpy as np

In [2]:
def calculate_h_index(citation_count_list):
    citation_count_list = sorted(citation_count_list, reverse = True)
    if len(citation_count_list) == 0:
        return 0
    for i in range(1, len(citation_count_list) + 1, 1):
        if i > citation_count_list[i - 1]:
            return i - 1
    return i

In [3]:
authors = pd.read_csv("results/kdd_authors.csv") 
authors

,Unnamed: 0,AuthorId,Rank,NormalizedName,DisplayName,LastKnownAffiliationId,PaperCount,CitationCount,CreateDate
0,0,757571,14925,ivan marsic,Ivan Marsic,1.023221e+08,159,1851,2016-06-24
1,1,798163,14299,carsten wiuf,Carsten Wiuf,1.240557e+08,172,7680,2016-06-24
2,2,941590,15655,ramasamy uthurusamy,Ramasamy Uthurusamy,1.181366e+08,30,1097,2016-06-24
3,3,989270,14621,edgar weippl,Edgar Weippl,1.458471e+08,308,2579,2016-06-24
4,4,1682464,17625,guy flysher,Guy Flysher,1.291425e+09,5,236,2016-06-24
...,...,...,...,...,...,...,...,...,...
7147,7147,2977264633,17717,karel slavicek,Karel Slavicek,2.144926e+07,13,56,2019-10-10
7148,7148,2977389243,17648,michal javornik,Michal Javornik,2.144926e+07,14,57,2019-10-10
7149,7149,2977831325,15559,alexey svyatkovskiy,Alexey Svyatkovskiy,2.008984e+07,214,236,2019-10-10
7150,7150,2978529943,18039,rafiqul islam,Rafiqul Islam,1.355989e+08,15,63,2019-10-10


In [4]:
paper_authors_affiliation = pd.read_csv("results/kdd_paper_author_affiliations_for_authors.csv") 
print(len(paper_authors_affiliation))
paper_authors_affiliation.head()

465746


,Unnamed: 0,PaperId,AuthorId,AffiliationId,AuthorSequenceNumber,OriginalAuthor,OritinalAffiliation
0,0,20062,193994343,NaN,2,Silvia Miksch,NaN
1,1,20062,1666292634,NaN,1,Wolfgang Aigner,NaN
2,2,24647,2305179268,NaN,1,Tadashi Okoshi,NaN
3,3,45737,2133330932,102913810.0,1,Xiuqin Ma,Universiti Malaysia Pahang
4,4,45737,2144708694,102913810.0,3,Hongwu Qin,Universiti Malaysia Pahang


In [5]:
PAPERS = "mag/Papers.txt"
PAPERS_HEADER = [
    "PaperId",
    "Rank",
    "Doi",
    "DocType",
    "PaperTitle",
    "OriginalTitle",
    "BookTitle",
    "Year",
    "Date",
    "Publisher",
    "JournalId",
    "ConferenceSeriesId",
    "ConferenceInstanceId",
    "Volumne",
    "Issue",
    "FirstPage",
    "LastPage",
    "ReferenceCount",
    "CitationCount",
    "EstimatedCitation",
    "OriginalVenue",
    "FamilyId",
    "CreatedDate",
]

def extract_author_papers(paper_authors_affiliation, num_loops=-1):
    chunksize = 10 ** 6
    count = 0
    relevantPapers = pd.DataFrame(columns=PAPERS_HEADER)
    for chunk in pd.read_csv(
        PAPERS,
        chunksize=chunksize,
        sep="\t",
        header=None,
        names=PAPERS_HEADER,
        low_memory=False,
    ):
        count += 1
        relevantPapers = relevantPapers.append(
            chunk[
                chunk.PaperId.isin(
                    list(paper_authors_affiliation.PaperId)
                )
            ],
            ignore_index=True,
        )
        if count == num_loops:
            break
    relevantPapers.to_csv(r"results/author_papers.csv")

In [6]:
extract_author_papers(paper_authors_affiliation)
author_papers = pd.read_csv("results/author_papers.csv") 
print(len(author_papers))
author_papers.head()

/home/ec2-user/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


355056


,Unnamed: 0,PaperId,Rank,Doi,DocType,PaperTitle,OriginalTitle,BookTitle,Year,Date,...,Volumne,Issue,FirstPage,LastPage,ReferenceCount,CitationCount,EstimatedCitation,OriginalVenue,FamilyId,CreatedDate
0,0,15354235,20380,10.1007/978-3-662-44777-2_60,Conference,document retrieval on repetitive collections,Document Retrieval on Repetitive Collections,ESA,2014,2014-09-08,...,NaN,NaN,725,736,18.0,5.0,5.0,european symposium on algorithms,1.535424e+07,2016-06-24
1,1,2950265543,20380,NaN,Repository,document retrieval on repetitive collections,Document Retrieval on Repetitive Collections,NaN,2014,2014-04-19,...,NaN,NaN,NaN,NaN,13.0,4.0,4.0,arXiv: Data Structures and Algorithms,1.535424e+07,2019-06-27
2,2,2949713162,22037,NaN,NaN,clustering of web users using the tensor decom...,Clustering of web users using the tensor decom...,NaN,2010,2010-06-01,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,Faculty of Science and Technology; Smart Servi...,6.771643e+07,2019-06-27
3,3,67716430,22037,NaN,Conference,clustering of web users using the tensor decom...,Clustering of web users using the tensor decom...,NaN,2010,2010-06-01,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,"international conference on user modeling, ada...",6.771643e+07,2016-06-24
4,4,1486066286,19174,NaN,Conference,identification of joint interventional distrib...,Identification of joint interventional distrib...,AAAI,2006,2006-07-16,...,NaN,NaN,1219,1226,14.0,80.0,97.0,national conference on artificial intelligence,1.486066e+09,2016-06-24


In [7]:
merged_dataframe = pd.concat([paper_authors_affiliation, author_papers], axis=1)
print(len(merged_dataframe))
merged_dataframe.head()

465746


,Unnamed: 0,PaperId,AuthorId,AffiliationId,AuthorSequenceNumber,OriginalAuthor,OritinalAffiliation,Unnamed: 0,PaperId,Rank,...,Volumne,Issue,FirstPage,LastPage,ReferenceCount,CitationCount,EstimatedCitation,OriginalVenue,FamilyId,CreatedDate
0,0,20062,193994343,NaN,2,Silvia Miksch,NaN,0.0,1.535424e+07,20380.0,...,NaN,NaN,725,736,18.0,5.0,5.0,european symposium on algorithms,1.535424e+07,2016-06-24
1,1,20062,1666292634,NaN,1,Wolfgang Aigner,NaN,1.0,2.950266e+09,20380.0,...,NaN,NaN,NaN,NaN,13.0,4.0,4.0,arXiv: Data Structures and Algorithms,1.535424e+07,2019-06-27
2,2,24647,2305179268,NaN,1,Tadashi Okoshi,NaN,2.0,2.949713e+09,22037.0,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,Faculty of Science and Technology; Smart Servi...,6.771643e+07,2019-06-27
3,3,45737,2133330932,102913810.0,1,Xiuqin Ma,Universiti Malaysia Pahang,3.0,6.771643e+07,22037.0,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,"international conference on user modeling, ada...",6.771643e+07,2016-06-24
4,4,45737,2144708694,102913810.0,3,Hongwu Qin,Universiti Malaysia Pahang,4.0,1.486066e+09,19174.0,...,NaN,NaN,1219,1226,14.0,80.0,97.0,national conference on artificial intelligence,1.486066e+09,2016-06-24


In [8]:
result = []
for idx, row in authors.iterrows():
    authorName = row.DisplayName
    authorId = row.AuthorId
    citations = list(merged_dataframe[merged_dataframe.AuthorId == authorId].CitationCount)
    citations = [x for x in citations if not np.isnan(x)]
    h_index = calculate_h_index(citations)
    result.append([authorId, authorName, h_index])
    
result_dataframe = pd.DataFrame(result)
result_dataframe.to_csv(r"results/hindex.csv")

In [9]:
result_dataframe

,0,1,2
0,757571,Ivan Marsic,27
1,798163,Carsten Wiuf,27
2,941590,Ramasamy Uthurusamy,12
3,989270,Edgar Weippl,36
4,1682464,Guy Flysher,3
...,...,...,...
7147,2977264633,Karel Slavicek,5
7148,2977389243,Michal Javornik,6
7149,2977831325,Alexey Svyatkovskiy,4
7150,2978529943,Rafiqul Islam,2
